In [ ]:
import numpy as np
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects

import rpy2.robjects.numpy2ri

In [ ]:
from DAX.HelpFunctions.get_dax_data import get_dax_data

df = get_dax_data()

In [ ]:
y = df['ret1'].dropna().to_numpy()
y

# Setup Arma Garch 1,1

In [ ]:
rpy2.robjects.numpy2ri.activate()

rugarch = rpackages.importr('rugarch')

#GARCH(1,1)
variance_model = robjects.ListVector({'model': "sGARCH",
                                      'garchOrder': robjects.IntVector([1, 1])})

#ARMA(1,1)
mean_model = robjects.ListVector({'armaOrder': robjects.IntVector([1, 1]),
                                  'include.mean': True})


#defines the DoF for the t-Dist
params= robjects.ListVector({'shape': 3})


model = rugarch.ugarchspec(variance_model=variance_model,
                           mean_model=mean_model,
                           distribution_model="std", # "std" -> student t, "norm" -> normal
                           fixed_pars=params)


#In the following, I'm assuming that your timeseries is in a numpy array called "y"
modelfit = rugarch.ugarchfit(spec=model,
                             data=y,
                             solver='hybrid',
                             tol=1e-3)

#historic mu and sigma
sigma_hist = np.asarray(rugarch.sigma(modelfit))
mu_hist = np.asarray(rugarch.fitted(modelfit))

#1-day ahead forecast mu and sigma
forecast = rugarch.ugarchforecast(modelfit)
sigma = np.asarray(rugarch.sigma(forecast))[0]
mu = np.asarray(rugarch.fitted(forecast))[0]

In [33]:
from DAX.Models import arma_garch_11 as ag11
import numpy as np
from DAX.HelpFunctions.get_dax_data import get_dax_data

import importlib
importlib.reload(ag11)

df = get_dax_data()
ag11.arma_garch_11(df)

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-13,DAX,1 day,-3.180269,-0.720820,0.057328,0.835475,3.294924
1,2023-12-14,DAX,2 day,-3.184259,-0.631431,0.176261,0.983952,3.536780
2,2023-12-15,DAX,5 day,-4.094467,-0.704449,0.368123,1.440694,4.830712
3,2023-12-18,DAX,6 day,-4.708343,-0.866121,0.349523,1.565167,5.407389
4,2023-12-19,DAX,7 day,-4.514567,-0.622331,0.609138,1.840606,5.732843


In [ ]:
df.tail(10)